<a href="https://colab.research.google.com/github/Ananassio/Data-Science_lab/blob/main/Project/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A first look at the data

In [1]:
#get data from server
!wget --no-check-certificate https://www.keuper-labs.org/project_ss22.zip

--2022-05-11 13:53:20--  https://www.keuper-labs.org/project_ss22.zip
Resolving www.keuper-labs.org (www.keuper-labs.org)... 91.250.118.129
Connecting to www.keuper-labs.org (www.keuper-labs.org)|91.250.118.129|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 70859177 (68M) [application/zip]
Saving to: ‘project_ss22.zip’

project_ss22.zip    100%[===================>]  67.58M  11.4MB/s    in 6.6s    

2022-05-11 13:53:27 (10.2 MB/s) - ‘project_ss22.zip’ saved [70859177/70859177]



In [2]:
#unpack
!unzip project_ss22.zip

Archive:  project_ss22.zip
   creating: dressipi_recsys2022/
  inflating: dressipi_recsys2022/candidate_items.csv  
  inflating: dressipi_recsys2022/item_features.csv  
  inflating: dressipi_recsys2022/test_leaderboard_sessions.csv  
  inflating: dressipi_recsys2022/test_final_sessions.csv  
  inflating: dressipi_recsys2022/README.txt  
  inflating: dressipi_recsys2022/train_purchases.csv  
  inflating: dressipi_recsys2022/train_sessions.csv  
  inflating: dressipi_recsys2022/README_win.txt  


In [3]:
import pandas as pd
import numpy as np

In [4]:
items = pd.read_csv('dressipi_recsys2022/candidate_items.csv')
features = pd.read_csv('dressipi_recsys2022/item_features.csv')
train_purchases = pd.read_csv('dressipi_recsys2022/train_purchases.csv')
train_sessions = pd.read_csv('dressipi_recsys2022/train_sessions.csv')
test_leaderboard_sessions = pd.read_csv('dressipi_recsys2022/test_leaderboard_sessions.csv')

## have a first look

In [5]:
items.head()

,item_id
0,4
1,8
2,9
3,19
4,20


In [6]:
#how many items do we have?
items.describe()

,item_id
count,4990.000000
mean,14007.035271
std,8218.231425
min,4.000000
25%,6833.500000
50%,14108.500000
75%,21200.000000
max,28137.000000


In [7]:
features.head(100)

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75
...,...,...,...
95,8,17,378
96,8,19,700
97,8,11,735
98,8,63,861


In [8]:
features[features['feature_category_id'] == 30]

,item_id,feature_category_id,feature_value_id
129,10,30,564
134,10,30,482
153,11,30,847
154,11,30,478
156,11,30,715
...,...,...,...
471557,28133,30,482
471561,28133,30,194
471590,28135,30,564
471593,28135,30,478


In [9]:
features[:100].pivot(index='item_id',columns='feature_category_id')

feature_value_id                                           \
feature_category_id               3      4      5      7      11     15    16   
item_id                                                                         
2                                NaN    NaN    NaN  394.0    NaN    NaN  38.0   
3                              889.0  618.0  605.0  452.0  859.0    NaN   NaN   
4                              793.0  618.0  605.0  837.0    NaN    NaN   NaN   
7                                NaN    NaN    NaN  536.0    NaN  133.0   NaN   
8                              793.0  618.0  605.0  798.0  735.0    NaN   NaN   

                                          ...                              \
feature_category_id     17     18     19  ...     56     59     61     62   
item_id                                   ...                               
2                      NaN    NaN    NaN  ...  365.0    NaN  462.0  801.0   
3                    378.0  817.0  254.0  ...  365.0  180.0  706.0    NaN   
4                    378.0  289.0  148.0  ...  365.0  180.0  462.0    NaN   
7                      NaN    NaN   33.0  ...  153.0    NaN  706.0    NaN   
8                    378.0  817.0  700.0  ...  365.0  856.0  462.0    NaN   

                                                              
feature_category_id     63     65     68     69    72     73  
item_id                                                       
2                      NaN    NaN  351.0  885.0  75.0    NaN  
3                    861.0  521.0   14.0  592.0  75.0  544.0  
4                    861.0  521.0  373.0  538.0  75.0  544.0  
7                    816.0    NaN  739.0  592.0  75.0    NaN  
8                    861.0  521.0  351.0  592.0  75.0  544.0  

[5 rows x 33 columns]

In [10]:
dup = features.duplicated()

In [11]:
dup.value_counts()

False    471751
dtype: int64

because there are some duplicated indexes, the mean index was used to generate the mean value of the feature value ids

In [12]:
features.pivot_table(index='item_id',columns='feature_category_id', aggfunc='mean')

feature_value_id                                        \
feature_category_id               1     2      3      4      5   6      7    
item_id                                                                      
2                                NaN   NaN    NaN    NaN    NaN NaN  394.0   
3                                NaN   NaN  889.0  618.0  605.0 NaN  452.0   
4                                NaN   NaN  793.0  618.0  605.0 NaN  837.0   
7                                NaN   NaN    NaN    NaN    NaN NaN  536.0   
8                                NaN   NaN  793.0  618.0  605.0 NaN  798.0   
...                              ...   ...    ...    ...    ...  ..    ...   
28139                            NaN   NaN  793.0  618.0  605.0 NaN  798.0   
28140                            NaN  53.0    NaN    NaN    NaN NaN    NaN   
28141                          461.0   NaN  889.0  719.0  605.0 NaN    2.0   
28142                            NaN   NaN    NaN    NaN    NaN NaN  619.0   
28143                            NaN   NaN    NaN    NaN    NaN NaN  798.0   

                                 ...                                        \
feature_category_id  8   9   10  ...    64     65  66     67     68     69   
item_id                          ...                                         
2                   NaN NaN NaN  ...   NaN    NaN NaN    NaN  351.0  885.0   
3                   NaN NaN NaN  ...   NaN  521.0 NaN    NaN   14.0  592.0   
4                   NaN NaN NaN  ...   NaN  521.0 NaN    NaN  373.0  538.0   
7                   NaN NaN NaN  ...   NaN    NaN NaN    NaN  739.0  592.0   
8                   NaN NaN NaN  ...   NaN  521.0 NaN    NaN  351.0  592.0   
...                  ..  ..  ..  ...   ...    ...  ..    ...    ...    ...   
28139               NaN NaN NaN  ...   NaN  521.0 NaN    NaN  106.0  805.0   
28140               NaN NaN NaN  ...  80.0    NaN NaN  349.0  351.0    NaN   
28141               NaN NaN NaN  ...   NaN    NaN NaN    NaN  379.0  499.0   
28142               NaN NaN NaN  ...   NaN  610.0 NaN    NaN  895.0  740.0   
28143               NaN NaN NaN  ...   NaN    NaN NaN    NaN  351.0  499.0   

                                              
feature_category_id  70     71     72     73  
item_id                                       
2                   NaN    NaN   75.0    NaN  
3                   NaN    NaN   75.0  544.0  
4                   NaN    NaN   75.0  544.0  
7                   NaN    NaN   75.0    NaN  
8                   NaN    NaN   75.0  544.0  
...                  ..    ...    ...    ...  
28139               NaN    NaN   75.0  544.0  
28140               NaN  226.0    NaN  544.0  
28141               NaN    NaN   75.0  544.0  
28142               NaN    NaN   75.0   91.0  
28143               NaN    NaN  371.0   91.0  

[23691 rows x 73 columns]

In [13]:
features["item_id"].describe()

count    471751.000000
mean      14058.539477
std        8107.465455
min           2.000000
25%        7060.000000
50%       14045.000000
75%       21063.000000
max       28143.000000
Name: item_id, dtype: float64

In [15]:
train_purchases.head()

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114


In [16]:
train_sessions.head()

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [17]:
test_leaderboard_sessions

,session_id,item_id,date
0,26,19185,2021-06-16 09:53:54.158
1,200,17089,2021-06-25 12:23:40.811
2,200,17089,2021-06-25 12:24:36.631
3,200,8060,2021-06-25 12:24:41.677
4,200,4758,2021-06-25 12:24:50.692
...,...,...,...
229349,4439653,25955,2021-06-11 10:22:57.47
229350,4439653,12179,2021-06-11 10:23:00.663
229351,4439757,2078,2021-06-30 11:42:15.073
229352,4439757,2078,2021-06-30 11:43:13.725


In [18]:
test_sessions= pd.DataFrame(columns=['session_id']);

In [19]:
test_sessions['session_id'] = test_leaderboard_sessions['session_id'].sort_values().unique()

In [20]:
test_sessions

,session_id
0,26
1,200
2,205
3,495
4,521
...,...
49995,4439446
49996,4439458
49997,4439550
49998,4439653


### Erstellen eines Testscores mit zufälligen ergebnissen

In [226]:
test = pd.DataFrame();

In [227]:
test['session_id'] = test_leaderboard_sessions['session_id'].unique()
test  = test.loc[test.index.repeat(100)].reset_index(drop=True)
test['item_id'] = pd.DataFrame(np.random.randint(train_purchases['item_id'].min(), train_purchases['item_id'].max(), size=(len(test),1)))
test['rank'] = test.groupby('session_id').cumcount()+1
test = test.set_index('session_id')


In [228]:
test.head(102)

,item_id,rank
session_id,,
26,23662,1
26,6064,2
26,26146,3
26,12745,4
26,22423,5
...,...,...
26,211,98
26,19269,99
26,6397,100


In [229]:
test.to_csv('test.csv')